In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import cv2
import numpy as np
# from google.colab.patches import cv2_imshow
import numpy as np
from sklearn.cluster import MiniBatchKMeans

In [ ]:
!unzip /content/drive/MyDrive/Inian_sign_language.zip

Streaming output truncated to the last 5000 lines.
  inflating: data/V/819.jpg          
  inflating: data/V/82.jpg           
  inflating: data/V/820.jpg          
  inflating: data/V/821.jpg          
  inflating: data/V/822.jpg          
  inflating: data/V/823.jpg          
  inflating: data/V/824.jpg          
  inflating: data/V/825.jpg          
  inflating: data/V/826.jpg          
  inflating: data/V/827.jpg          
  inflating: data/V/828.jpg          
  inflating: data/V/829.jpg          
  inflating: data/V/83.jpg           
  inflating: data/V/830.jpg          
  inflating: data/V/831.jpg          
  inflating: data/V/832.jpg          
  inflating: data/V/833.jpg          
  inflating: data/V/834.jpg          
  inflating: data/V/835.jpg          
  inflating: data/V/836.jpg          
  inflating: data/V/837.jpg          
  inflating: data/V/838.jpg          
  inflating: data/V/839.jpg          
  inflating: data/V/84.jpg           
  inflating: data/V/840.jpg          

In [ ]:
import os

dataset_path ="/content/data"
# List all the image file paths
image_paths = []
labels = []
for folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder)
    if os.path.isdir(folder_path):
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            image_paths.append(img_path)
            labels.append(folder)

In [ ]:
from sklearn.model_selection import train_test_split

# Split the image file paths and labels into train and test sets
train_paths, test_paths, train_labels, test_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)

# Print the number of images in each set
print(f"Number of training images: {len(train_paths)}")
print(f"Number of test images: {len(test_paths)}")

# # Select a subset of images from the train and test sets
# train_paths_subset = train_paths[:100]
# train_labels_subset = train_labels[:100]
# test_paths_subset = test_paths[:50]
# test_labels_subset = test_labels[:50]

# print(f"number of trainig images : {len(train_paths_subset)}")
# print(f"number of trainig labels : {len(train_labels_subset)}")
# print(f"number of test images : {len(test_paths_subset)}")
# print(f"number of test labels : {len(test_labels_subset)}")

Number of training images: 33600
Number of test images: 8400


In [ ]:
import numpy as np

# Count the number of occurrences of each label in the training set
unique_labels, counts = np.unique(test_labels, return_counts=True)

# Print the number of occurrences of each label
print("Number of occurrences of each label in the training set:")
for label, count in zip(unique_labels, counts):
    print(f"{label}: {count}")

# Print the classes
print("Classes:")
print(np.unique(labels))


Number of occurrences of each label in the training set:
1: 254
2: 263
3: 234
4: 241
5: 230
6: 252
7: 223
8: 240
9: 237
A: 224
B: 248
C: 241
D: 234
E: 235
F: 239
G: 239
H: 210
I: 265
J: 243
K: 213
L: 243
M: 258
N: 262
O: 227
P: 224
Q: 240
R: 239
S: 235
T: 230
U: 254
V: 263
W: 254
X: 236
Y: 248
Z: 222
Classes:
['1' '2' '3' '4' '5' '6' '7' '8' '9' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I'
 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']


In [ ]:
def extract_features(img_path):
    # Read the image
    img = cv2.imread(img_path)

    # Apply skin masking
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    result = cv2.bitwise_and(img, img, mask=mask)

    # Convert image to grayscale
    gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

    # Create SIFT object and find keypoints and descriptors
    sift = cv2.SIFT_create()
    kp, des = sift.detectAndCompute(gray, None)

    return des


In [ ]:
X_train = []
y_train = []

for i, img_path in enumerate(train_paths):
    # Extract features using the function
    des = extract_features(img_path)
    
    # Save the SIFT descriptors and corresponding label
    X_train.append(des)
    y_train.append(train_labels[i])



In [ ]:
print(len(X_train))
print(len(y_train))

33600
33600


In [ ]:
# Flatten the SIFT feature vectors
X_train_flat = np.concatenate(X_train)

# Create Mini Batch K-Means object
mbk = MiniBatchKMeans(n_clusters=50, random_state=0, batch_size=100)

# Fit the Mini Batch K-Means model to the SIFT features
mbk.fit(X_train_flat)

# Calculate the histogram of visual words for each image
X_train_hist = []
for des in X_train:
    pred = mbk.predict(des)
    hist, _ = np.histogram(pred, bins=range(51))
    X_train_hist.append(hist)

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
from sklearn import svm

# Train the SVM model
clf = svm.SVC(kernel='linear')
clf.fit(X_train_hist, y_train)

SVC(kernel='linear')

In [ ]:
import sklearn
print(sklearn.__version__)


1.2.2


In [ ]:
import joblib
# Save the trained models
joblib.dump(mbk, 'mbk_model_final.pkl')


['mbk_model_final.pkl']

In [ ]:
joblib.dump(clf, 'svm_model_final.pkl')

['svm_model_final.pkl']

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from joblib import load

# Load the trained models
mbk = load('mbk_model_final.pkl')
clf = load('svm_model_final.pkl')


X_test = []
y_test = []

for i, img_path in enumerate(test_paths):
    # Extract features using the function
    des = extract_features(img_path)
    
    # Save the SIFT descriptors and corresponding label
    X_test.append(des)
    y_test.append(test_labels[i])

X_test_flat = np.concatenate(X_test)

# Predict visual words for test images
X_test_hist = []
for des in X_test:
    pred = mbk.predict(des)
    hist, _ = np.histogram(pred, bins=range(51))
    X_test_hist.append(hist)

# Perform predictions using the trained SVM model
y_pred = clf.predict(X_test_hist)


In [ ]:
class_names = ['1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [ ]:
from sklearn.metrics import accuracy_score

# Calculate overall accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Overall Accuracy:", accuracy)


Overall Accuracy: 0.996547619047619


In [ ]:
# Calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)


Confusion Matrix:
[[252   0   0 ...   0   0   0]
 [  0 262   1 ...   0   0   0]
 [  0   3 230 ...   0   0   0]
 ...
 [  0   0   0 ... 236   0   0]
 [  0   0   0 ...   0 248   0]
 [  0   0   0 ...   0   0 222]]


In [ ]:
# Calculate classification report
classification_rep = classification_report(y_test, y_pred,target_names=class_names)
print("Classification Report:")
print(classification_rep)


Classification Report:
              precision    recall  f1-score   support

           1       0.99      0.99      0.99       254
           2       0.98      1.00      0.99       263
           3       0.99      0.98      0.99       234
           4       1.00      0.99      0.99       241
           5       0.99      1.00      0.99       230
           6       1.00      1.00      1.00       252
           7       1.00      1.00      1.00       223
           8       1.00      1.00      1.00       240
           9       1.00      1.00      1.00       237
           A       1.00      1.00      1.00       224
           B       1.00      1.00      1.00       248
           C       0.99      1.00      0.99       241
           D       1.00      1.00      1.00       234
           E       1.00      1.00      1.00       235
           F       1.00      1.00      1.00       239
           G       1.00      1.00      1.00       239
           H       1.00      1.00      1.00       210
    

In [ ]:
X_test = []
y_test = []

for i, img_path in enumerate(test_paths):
    # Extract features using the function
    des = extract_features(img_path)
    
    # Save the SIFT descriptors and corresponding label
    X_test.append(des)
    y_test.append(test_labels[i])

# Flatten the SIFT feature vectors
X_test_flat = np.concatenate(X_test)

# Calculate the histogram of visual words for each image
X_test_hist = []
for des in X_test:
    pred = mbk.predict(des)
    hist, _ = np.histogram(pred, bins=range(51))
    X_test_hist.append(hist)

# Use the trained SVM classifier to predict the labels for the test set
y_pred = clf.predict(X_test_hist)

# Print the predicted labels and the true labels for the test set
print("Predicted labels:", y_pred)
# # print("True labels:", y_test)

# # Stack the true labels and predicted labels horizontally
# labels = np.column_stack((np.array(y_test), y_pred))

# # Print the labels
# print(labels)


Predicted labels: ['E' 'Q' 'K' ... 'C' 'M' 'Y']


In [ ]:
# import cv2
# import joblib
# import numpy as np

# # Load the trained models
# mbk_model = joblib.load('mbk_model_final.pkl')
# svm_model = joblib.load('svm_model_final.pkl')

# cam = cv2.VideoCapture(0)
# while True:
#     ret, frame = cam.read()
#     if not ret:
#         print("Failed to read frame from camera")
#         break
#     frame = cv2.flip(frame, 1)
#     cv2.rectangle(frame, (319, 9), (620 + 1, 309), (0, 255, 0), 1)
#     roi = frame[10:300, 320:620]
    
#     # cv2.imshow("Frame", frame)
#     # des = extract_features(roi)
#     hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
#     lower_skin = np.array([0, 20, 70], dtype=np.uint8)
#     upper_skin = np.array([20, 255, 255], dtype=np.uint8)
#     mask = cv2.inRange(hsv, lower_skin, upper_skin)
#     kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
#     mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
#     result = cv2.bitwise_and(roi, roi, mask=mask)

#     # Convert image to grayscale
#     gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

#     # Create SIFT object and find keypoints and descriptors
#     sift = cv2.SIFT_create()
#     kp, des = sift.detectAndCompute(gray, None)
#     # pred = mbk.predict(des)
#     # hist, _ = np.histogram(pred, bins=range(51))
#     # # y_pred = clf.predict(hist)
#     # y_pred = clf.predict(hist.reshape(1, -1))
#     if des is not None:
#         pred = mbk_model.predict(des)
#         hist, _ = np.histogram(pred, bins=range(51))
#         y_pred = svm_model.predict(hist.reshape(1, -1))
#         cv2.putText(frame, str(y_pred[0]), (10, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)


    # # print(alpha_dict[np.argmax(pred)])
    # # cv2.putText(frame,y_pred, (10, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
    # # cv2.putText(frame, str(y_pred[0]), (10, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
    # cv2.imshow("Frame", frame)
    # k = cv2.waitKey(100) & 0xFF
    # if k == 27:
    #     break

# cam.release()
# cv2.destroyAllWindows()

    